In [ ]:
import pandas as pd
from google.colab import files

# Load the raw data
df = pd.read_csv("data.csv")

# 1. Drop exact duplicate rows
df = df.drop_duplicates()

# 2. Drop duplicate complaints based on 'File No.'
df = df.drop_duplicates(subset=['File No.'], keep='first')

# 3. Group rare categories in categorical columns under 'Other'
def group_rare_categories(df, threshold=0.02):
    """
    Replaces rare categories (below threshold) in object columns with 'Other'.
    """
    categorical_cols = df.select_dtypes(include=['object']).columns
    for col in categorical_cols:
        freq = df[col].value_counts(normalize=True)
        rare = freq[freq < threshold].index
        df[col] = df[col].replace(rare, 'Other')
    return df

df = group_rare_categories(df)

# 4. Convert 'Recovery' to numeric if it's not already
df['Recovery'] = pd.to_numeric(df['Recovery'], errors='coerce')

# 5. Drop rows that are completely empty (if any)
df = df.dropna(how='all')

# 6. Save the cleaned DataFrame
df.to_csv("final_cleaned.csv", index=False)
print("✅ Cleaned CSV saved as 'final_cleaned.csv'")
files.download("final_cleaned.csv")

✅ Cleaned CSV saved as 'final_cleaned.csv'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Why the Cleaning Code Works:

**1. Rare Category Grouping**

Our insurance data has tons of random company names, subcoverages, and disposition outcomes.
Grouping anything that appears in <2% of the data into 'Other' keeps our model from getting overwhelmed by noise.


**2. Duplicates (based on 'File No.') Removal:**

Complaints might be logged multiple times or appear with slightly different info.
Dropping extras avoids inflating trends.


**3. Keeping Outliers (Recovery):**

In insurance, outliers are the story.
Settlements are either:

0 dollars or up to 843,000 dollars in settlements.

**For regression:** the outliers are our targets

**Doing anomaly detection:** the outliers are the anomalies

**clustering:** the outliers will shift clusters, which is fine for our usage.
-> In clustering (like k-means), outliers can distort how clusters are formed.
But in our case — insurance data — that’s not a bad thing.

Example:

A bunch of 0 or $100 settlements might group together.

But a $500,000+ recovery could form its own cluster — that’s real signal, not noise.

So instead of filtering them out, we let them stay, because they represent:

Large recoveries

Possibly complex or escalated complaints

Things our ML models should be aware of

We retained outliers in recovery amounts during clustering because extreme values are inherent to insurance claims. These high-recovery cases often reflect real-world escalation or rare scenarios and may naturally form distinct clusters. Removing them would obscure important patterns.
We should not remove outliers for this use case unless we're testing a “norm-only” model for comparison.

4. Keeping Missing Data Imputation Later:
We didn’t impute here, which is actually great for us because our notebook will handle it with logic later:

E.X:

50% missing → drop the column

mean/Unknown fill elsewhere


this cleaned CSV isn’t “perfect” in the textbook sense — but it’s tailored for insurance ML workflows, which is more important.
We preserved important variation and didn’t over-clean, which a lot of people mess up.
